In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors 

df_airbnb = pd.read_csv('Airbnb_Open_Data.csv')

/var/folders/5v/s327w09x2b7bc679clc46r0h0000gq/T/ipykernel_69269/1412995961.py:6: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_airbnb = pd.read_csv('Airbnb_Open_Data.csv')


In [37]:
# drop the column we do not use
df_airbnb = df_airbnb.drop(['id', 'host id','country','country code','license','last review'], axis=1)

# change host_identity_verified to binary int
df_airbnb['host_identity_verified'] = df_airbnb['host_identity_verified'].map({'verified': 1, 'unconfirmed': 0})

# combine the same group for neighbourhood group
df_airbnb['neighbourhood group'] = df_airbnb['neighbourhood group'].replace('brookln', 'Brooklyn').replace('manhatan', 'Manhattan')

# Fill missing values in the 'neighbourhood group' column with a placeholder (e.g., 'Unknown')
df_airbnb['neighbourhood group'].fillna('Unknown', inplace=True)
df_airbnb['neighbourhood'].fillna('Unknown', inplace=True)

# Apply the function to fill missing prices
df_airbnb['price'] = df_airbnb['price'].replace('[\$,]', '', regex=True)
df_airbnb['service fee'] = df_airbnb['service fee'].replace('[\$,]', '', regex=True)

# Convert the "price" column to a float data type
df_airbnb['price'] = df_airbnb['price'].astype(float)
df_airbnb['service fee'] = df_airbnb['service fee'].astype(float)

# Calculate the average price for each neighborhood
average_prices = df_airbnb.groupby('neighbourhood')['price'].mean()

# Function to fill missing prices with the average price of the neighborhood
def fill_missing_price(row):
    if pd.isnull(row['price']):
        return average_prices[row['neighbourhood']]
    else:
        return row['price']

# Apply the function to update missing prices
df_airbnb['price'] = df_airbnb.apply(fill_missing_price, axis=1)

#Apply latitude and longitude restrictions for NYC
df_airbnb = df_airbnb[(df_airbnb['lat']>40.55) &(df_airbnb['lat']<40.94) & (df_airbnb['long']>-74.1) & (df_airbnb['long']<-73.67)]

#Deal with Strings: 
def df_normalize_strings(df, col):
    df[col] = df[col].str.lower() # change everything to lower cases
    df[col] = df[col].fillna(np.nan) # make None -> np.nan
    df[col] = df[col].replace('none or unspecified', np.nan)
    df[col] = df[col].replace('none', np.nan)
    df[col] = df[col].replace('no', np.nan)
    df[col] = df[col].replace('#name?', np.nan)
    df[col] = df[col].replace('', np.nan) # empty -> NaN

df_airbnb['NAME'] = df_airbnb['NAME'].astype(str)
df_airbnb['house_rules'] = df_airbnb['house_rules'].astype(str)
df_normalize_strings(df_airbnb, 'NAME')
df_normalize_strings(df_airbnb, 'house_rules')

#Change to catgorical 
def df_to_categorical(df, col):
    df[col] = df[col].astype('category')

df_to_categorical(df_airbnb, 'neighbourhood group')
df_to_categorical(df_airbnb, 'neighbourhood')
df_to_categorical(df_airbnb, 'instant_bookable')
df_to_categorical(df_airbnb, 'cancellation_policy')
df_to_categorical(df_airbnb, 'room type')



In [38]:
df_airbnb.dtypes

NAME                                object
host_identity_verified             float64
host name                           object
neighbourhood group               category
neighbourhood                     category
lat                                float64
long                               float64
instant_bookable                  category
cancellation_policy               category
room type                         category
Construction year                  float64
price                              float64
service fee                        float64
minimum nights                     float64
number of reviews                  float64
reviews per month                  float64
review rate number                 float64
calculated host listings count     float64
availability 365                   float64
house_rules                         object
dtype: object

In [39]:
df_airbnb.isnull().sum()

NAME                                 20
host_identity_verified              288
host name                           403
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
instant_bookable                    105
cancellation_policy                  76
room type                             0
Construction year                   213
price                                 0
service fee                         271
minimum nights                      408
number of reviews                   183
reviews per month                 15829
review rate number                  326
calculated host listings count      319
availability 365                    448
house_rules                        2695
dtype: int64